In [1]:
import os

In [16]:
os.environ['OPENAI_API_KEY']='your-openai-key'

In [3]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI 

In [4]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader



In [6]:
loader=PyPDFLoader("Indian Penal Code Book.pdf")
text=loader.load()

In [7]:
len(text)

1871

In [8]:
text


[Document(page_content='THE INDIAN PENAL CODE\nCHAPTER I INTRODUCTION\nThe Indian Penal Code was drafted by the First Indian Law Commission presided over\nby Lord Thomas Babington Macaulay. The draft underwent further revision at the hands\nof well-known jurists, like Sir Barnes Peacock, and was completed in 1850. The Indian\nPenal Code was passed by the then Legislature on 6 October 1860 and was enacted as\nAct No. XLV of 1860.\nPreamble. WHEREAS it is expedient to provide a general\nPenal Code for India; It is enacted as follows:—\nCOMMENT.—The Indian Penal Code, 1860 (IPC, 1860) exhaustively codiﬁes the law\nrelating to offences with which it deals and the rules of the common law cannot be\nresorted to for inventing exemptions which are not expressly enacted.1. It is not\nnecessary and indeed not permissible to construe the IPC, 1860 at the present day in\naccordance with the notions of criminal jurisdiction prevailing at the time when the\nCode was enacted. The notions relating to 

In [9]:
loader=DirectoryLoader("",glob="ipctextcorpus.txt",loader_cls=TextLoader)

In [10]:
document=loader.load()

In [16]:
len(document)

1872

In [15]:
document=document+text

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

In [19]:
text=text_splitter.split_documents(document)

In [20]:
text

[Document(page_content='1 \n THE INDIAN PENAL CODE  \n______________  \nARRANGEMENT OF SECTIONS  \n_________________  \nCHAPTER I  \nINTRODUCTION  \nPREAMBLE  \nSECTIONS  \n1. Title and extent of operation of the Code.  \n2. Punishment of offences committed within India.  \n3. Punishment of offences committed beyond, but  which by law may be tried within, India.  \n4. Extension of Code to extra -territorial offences.  \n5. Certain laws not to be affected by this Act.  \nCHAPTER II  \nGENERAL  EXPLANATIONS  \n6. Definitions in the Code to be understood subject to exceptions.  \n7. Sense of expression once  explained.  \n8. Gender.  \n9. Number.  \n10. “Man”.  “Woman”.  \n11. “Person ”. \n12.  “Public”.  \n13. [Omitted .] \n14. “Servant of Government”.  \n15. [Repealed.]  \n16. [Repealed.]  \n17. “Government”.  \n18. “India”.  \n19. “Judge”.  \n20. “Court of Justice”.  \n21. “Public servant”.  \n22. “Moveable property”.  \n23. “Wrongful gain”.  \n“Wrongful loss”.  \nGainin g wrongfully /

In [21]:
len(text)

6709

In [23]:
text[200]

Document(page_content='private, whether run by the Central Government, the State Government, local bodies or any other person, \ncontravenes the provisions of section 357C of the Code of Crim inal Procedure, 1973 (2 of 1974), shall be \npunished with imprisonment for a term which may extend to one year or with fine or with both.]  \n167. Public servant framing an incorrect documen t with intent to cause injury .—Whoever, being \na public servant, and be ing, as 2[such public servant, charged with the preparation or translation of any \ndocument  or electronic record , frames , prepares  or translates that document  or electronic record]  in a \nmanner which he knows or believes to be incorrect, intending thereby to cause or knowing it to be likely \nthat he may thereby cause injury to any person, shall be punished with imprisonment of either description \nfor a term which may extend to three ye ars, or with fine, or with both.', metadata={'source': 'ipctextcorpus.txt'})

# chroma db

In [14]:
# chromadb is local database

In [5]:
persist_directory='ip+pdf'

In [6]:
embedding = OpenAIEmbeddings()

In [34]:
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x1411c91d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x130619ad0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None)

In [35]:
vectordb=Chroma.from_documents(documents=text,
                               embedding=embedding,
                               persist_directory=persist_directory)

In [36]:
vectordb.persist()
vectordb=None

In [7]:
vectordb=Chroma(persist_directory=persist_directory,
                embedding_function=embedding)

/Users/aasarmehdi/.pyenv/versions/3.11.2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


# Make a retriever

In [8]:
retriver=vectordb.as_retriever()


In [53]:
docs=retriver.get_relevant_documents("punishment of murder?")

In [54]:
len(docs)

4

In [55]:
retriever=vectordb.as_retriever(search_kwargs={"k":2})

In [56]:
retriever.search_type

'similarity'

In [57]:
retriever.search_kwargs

{'k': 2}

In [33]:
docs=retriever.get_relevant_documents("punishment of murder?")

In [34]:
len(docs)

2

# Make a Chain

In [28]:
from langchain_community.llms import Ollama

llm=OpenAI(temperature=0.3,openai_api_key="your-openai-key")
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x15ff49790>, async_client=<openai.resources.completions.AsyncCompletions object at 0x16abbb110>, temperature=0.3, openai_api_key=SecretStr('**********'), openai_proxy='')

In [29]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

model = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [31]:
retriever.search_type

'similarity'

In [32]:
response=retrieval_chain.invoke({"input":"What are the provisions of the Indian Penal Code regarding theft?"})
response

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************1TQo. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [16]:
response['answer']

'\nAnswer: According to the Indian Penal Code, Chapter XVII, Section 379, whoever commits theft shall be punished with imprisonment of either description for a term which may extend to three years, or with fine, or with both. In order to constitute theft, five factors are essential: dishonest intention to take property, movable property, taking out of possession of another person, without their consent, and some movement of the property. Additionally, Section 380 states that theft in a dwelling house, tent, or vessel used as a human dwelling or for the custody of property, shall be punished with imprisonment for a term which may extend to seven years and a fine. Furthermore, Section 411 states that whoever dishonestly receives or retains stolen property, knowing or having reason to believe it to be stolen, shall be punished with imprisonment for a term which may extend to three years, or with fine, or with both. Finally, Section 418 states that whoever cheats with the knowledge that it

In [17]:
response=retrieval_chain.invoke({"input":"what legal issue can faced if accused for Rioting, armed with deadly weapon?"})

In [18]:
response['answer']

'\nAnswer: The accused may face imprisonment for a term of up to three years, a fine, or both, as stated in section 148 of the Indian Penal Code. Additionally, they may also face charges for being a member of an unlawful assembly, as mentioned in section 149, and for hiring or conniving at hiring persons to join an unlawful assembly, as stated in section 150. These charges may result in further penalties and consequences for the accused.'

In [85]:
response

{'input': 'Rioting, armed with deadly weapon?',
 'context': [Document(page_content='THE INDIAN PENAL CODE\nCHAPTER VIII OF OFFENCES AGAINST THE PUBLIC TRANQUILLITY\nThe offences in this chapter may be classiﬁed in the following four groups:—\nI. Unlawful assembly.\n(1)Being a member of an unlawful assembly (sections 141, 142, 143).\n(2)Joining an unlawful assembly armed with deadly weapons (section 144).\n(3)Joining or continuing in an unlawful assembly knowing it has been commanded\nto disperse (section 145).\n(4)Hiring of persons to join an unlawful assembly (section 150).\n(5)Harbouring persons hired for an unlawful assembly (section 157).\n(6)Being hired to take part in an unlawful assembly (section 158).\nII. Rioting (sections 146, 147).\n(1)Rioting with deadly weapon (section 148).\n(2)Assaulting or obstructing a public servant in the suppression of a riot (section\n152).\n(3)Wantonly giving provocation with intent to cause riot (section 153).\n(4)Liability of the owner or occupi

In [24]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Given the following context and a question, generate an answer based on this context only.
You're a chatbot specializing in Indian law, particularly the Indian Penal Code (IPC). Your user seeks guidance on legal matters within the scope of the IPC.
Engage with them in a helpful and informative manner, providing accurate answers to their questions and clarifications regarding relevant legal concepts ,statutes.
Ensure your responses are clear, concise, and tailored to the specific queries posed.


In the answer try to provide as much text as possible from the source document context .
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.
<context>
{context}
</context>
Question: {input}
"""
)
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=vectordb.as_retriever()
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [25]:
response=retrieval_chain.invoke({"input":"What are the provisions of the Indian Penal Code regarding theft?"})
response

{'input': 'What are the provisions of the Indian Penal Code regarding theft?',
 'context': [Document(page_content='THE INDIAN PENAL CODE\nCHAPTER XVII OF OFFENCES AGAINST PROPERTY\nOf Theft\n[s 380] Theft in dwelling house, etc.\nWhoever commits theft in any building,1 tent or vessel, which building, tent or vessel\nis used as a human dwelling, or used for the custody of property, shall be punished\nwith imprisonment of either description for a term which may extend to seven years,\nand shall also be liable to ﬁne.\nState Amendment\nTamil Nadu.—The following amendments were made by Tamil Nadu Act No. 28 of 1993,\nSection 2.\nSection 380 of the Indian Penal Code (Central Act XLV of 1860) (hereinafter in this Part\nreferred to as the principal Act), shall be renumbered as sub-section (1) of that section\nand after sub-section (1) as so renumbered, the following sub-section shall be added,\nnamely:—\n"(2) Whoever commits theft in respect of any idol or icon in any building used as a\nplac

In [26]:
response['answer']

'Answer: According to the Indian Penal Code, Chapter XVII, Section 379, whoever commits theft shall be punished with imprisonment of either description for a term which may extend to three years, or with fine, or with both. In order to constitute theft, five factors are essential: dishonest intention to take property, the property must be movable, the property should be taken out of the possession of another person, the property should be taken without the consent of that person, and there must be some moving of the property in order to accomplish the taking of it.'

In [27]:
response=retrieval_chain.invoke({"input":"Rioting, armed with deadly weapon?"})

In [28]:
response['answer']

'\nAnswer: Yes, rioting while being armed with a deadly weapon or anything that can be used as a weapon of offence is a punishable offense under section 148 of the Indian Penal Code. The punishment for this offense can be imprisonment for a term of up to three years, or a fine, or both. This is considered an aggravated form of the offense mentioned in the previous section, and the punishment is enhanced accordingly.'

In [91]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Given the following context and a question, generate an answer based on this context only.
You're a chatbot specializing in Indian law, particularly the Indian Penal Code (IPC). Your user seeks guidance on legal matters within the scope of the IPC.
Engage with them in a helpful and informative manner, providing accurate answers to their questions and clarifications regarding relevant legal concepts ,statutes and also punishment.
Ensure your responses are clear, concise, and tailored to the specific queries posed.


In the answer try to provide as much text as possible from "response" section in the source document context .
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.
<context>
{context}
</context>
Question: {input}
"""
)
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=vectordb.as_retriever()
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [92]:
response=retrieval_chain.invoke({"input":"What are the provisions of the Indian Penal Code regarding theft?"})
response

{'input': 'What are the provisions of the Indian Penal Code regarding theft?',
 'context': [Document(page_content='THE INDIAN PENAL CODE\nCHAPTER XVII OF OFFENCES AGAINST PROPERTY\nOf Theft\n[s 380] Theft in dwelling house, etc.\nWhoever commits theft in any building,1 tent or vessel, which building, tent or vessel\nis used as a human dwelling, or used for the custody of property, shall be punished\nwith imprisonment of either description for a term which may extend to seven years,\nand shall also be liable to ﬁne.\nState Amendment\nTamil Nadu.—The following amendments were made by Tamil Nadu Act No. 28 of 1993,\nSection 2.\nSection 380 of the Indian Penal Code (Central Act XLV of 1860) (hereinafter in this Part\nreferred to as the principal Act), shall be renumbered as sub-section (1) of that section\nand after sub-section (1) as so renumbered, the following sub-section shall be added,\nnamely:—\n"(2) Whoever commits theft in respect of any idol or icon in any building used as a\nplac

In [93]:
response['answer']

'Answer: According to section 379 of the Indian Penal Code, whoever commits theft shall be punished with imprisonment of either description for a term which may extend to three years, or with fine, or with both. In order to constitute theft, five factors are essential: dishonest intention to take property, the property must be movable, it should be taken out of the possession of another person, it should be taken without the consent of that person, and there must be some moving of the property in order to accomplish the taking of it.'

In [ ]:
# 'Answer: According to section 379 of the Indian Penal Code, whoever commits theft shall be punished with imprisonment, fine, or both. 
# To constitute theft, there must be a dishonest intention to take property, the property must be movable, it must be taken without the 
# consent of the owner, and there must be some movement of the property. Additionally, section 411 states that whoever dishonestly receives
# or retains stolen property shall also be punished. And under section 418, anyone who cheats with the knowledge of causing wrongful loss to
# a person whose interest they are bound to protect shall also face punishment.'


In [ ]:
# According to the Indian Penal Code, Chapter XVII, Section 379 and Section 380, whoever commits theft shall be punished with imprisonment
# for a term up to three years, or with fine, or both. In case of theft in a building, tent, or vessel used as a human dwelling or for custody 
# of property, the punishment can extend up to seven years of imprisonment and a fine. Additionally, under Section 411, whoever dishonestly 
# receives or retains any stolen property, knowing or having reason to believe the same to be stolen property, can be imprisoned
# for up to three years or fined.'

In [115]:
llm =Ollama(model="llama3")
from langchain_core.output_parsers import StrOutputParser
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Please responce to the user queries"),
        ("user","Question:{question}")
    ]
)
output_parser=StrOutputParser()
chain=prompt|llm|output_parser

In [116]:
chain.invoke({"question":"hii"})

KeyboardInterrupt: 

In [107]:
llm2.invoke("hii")

KeyboardInterrupt: 

In [109]:
response=retrieval_chain.invoke({"input":"What are the provisions of the Indian Penal Code regarding theft?"})
response

{'input': 'What are the provisions of the Indian Penal Code regarding theft?',
 'context': [Document(page_content='THE INDIAN PENAL CODE\nCHAPTER XVII OF OFFENCES AGAINST PROPERTY\nOf Theft\n[s 380] Theft in dwelling house, etc.\nWhoever commits theft in any building,1 tent or vessel, which building, tent or vessel\nis used as a human dwelling, or used for the custody of property, shall be punished\nwith imprisonment of either description for a term which may extend to seven years,\nand shall also be liable to ﬁne.\nState Amendment\nTamil Nadu.—The following amendments were made by Tamil Nadu Act No. 28 of 1993,\nSection 2.\nSection 380 of the Indian Penal Code (Central Act XLV of 1860) (hereinafter in this Part\nreferred to as the principal Act), shall be renumbered as sub-section (1) of that section\nand after sub-section (1) as so renumbered, the following sub-section shall be added,\nnamely:—\n"(2) Whoever commits theft in respect of any idol or icon in any building used as a\nplac

In [38]:
type(response['answer'])

str

In [ ]:
response

In [76]:
from langchain.chains import RetrievalQA

In [77]:
qa_chain=RetrievalQA.from_chain_type(llm=OpenAI(),
                                     chain_type="stuff",
                                     retriever=retriever,
                                     return_source_documents=True)

In [78]:
query="punishment of murder?"
llm_responce=qa_chain(query)
llm_responce

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': 'punishment of murder?',
 'result': ' The punishment for murder is either death or imprisonment for life and also a fine.',
 'source_documents': [Document(page_content='State of MP, (2008) 12 SCC 270 [LNIND 2008 SC 2798] .\nThe punishment for murder is either death or imprisonment for life and also ﬁne. The\nSupreme Court has, in a long course of decisions, made it an established principle that\nthe normal punishment for murder is life imprisonment and that "death" should be\nawarded in "rarest of rare cases". Examples of such rarest of rare cases are:\nLeading cases:—Kehar Singh v State (Delhi Administration). Lichhmadevi v State of\nRajasthan.\nAn abnormal delay in executing a death sentence has been recognised as a ground for\nconverting death sentence into life imprisonment.\nLeading cases:—Bachan Singh v State of Punjab Triveniben v State of Gujarat Madhu\nMehta v UOI.\nThe Supreme Court has noted serious changes which have taken place in the state of\nthe society since 

In [79]:
query="Rioting, armed with deadly weapon"
llm_responce=qa_chain(query)
llm_responce

{'query': 'Rioting, armed with deadly weapon',
 'result': ' Rioting with a deadly weapon is considered an offence against public tranquillity under the Indian Penal Code, specifically under section 148. This includes using a deadly weapon to participate in a riot or incite violence.',
 'source_documents': [Document(page_content='THE INDIAN PENAL CODE\nCHAPTER VIII OF OFFENCES AGAINST THE PUBLIC TRANQUILLITY\nThe offences in this chapter may be classiﬁed in the following four groups:—\nI. Unlawful assembly.\n(1)Being a member of an unlawful assembly (sections 141, 142, 143).\n(2)Joining an unlawful assembly armed with deadly weapons (section 144).\n(3)Joining or continuing in an unlawful assembly knowing it has been commanded\nto disperse (section 145).\n(4)Hiring of persons to join an unlawful assembly (section 150).\n(5)Harbouring persons hired for an unlawful assembly (section 157).\n(6)Being hired to take part in an unlawful assembly (section 158).\nII. Rioting (sections 146, 147).\

In [80]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\n Sources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [81]:
process_llm_response(llm_responce)

 Rioting with a deadly weapon is considered an offence against public tranquillity under the Indian Penal Code, specifically under section 148. This includes using a deadly weapon to participate in a riot or incite violence.


 Sources:
Indian Penal Code Book.pdf
Indian Penal Code Book.pdf
Indian Penal Code Book.pdf
Indian Penal Code Book.pdf


In [94]:
query = "What are the provisions of the Indian Penal Code regarding theft?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
     

 The Indian Penal Code has several sections that deal with theft. Section 379 states that whoever commits theft shall be punished with imprisonment of up to three years, or with a fine, or both. Section 380 specifies that if the theft is committed in a building, tent, or vessel that is used as a human dwelling or for the custody of property, the punishment can extend up to seven years of imprisonment and a fine. Additionally, Tamil Nadu has a state amendment that adds a sub-section (2) to Section 380, which states that theft of an idol or icon from a place of worship can lead to rigorous imprisonment for an unspecified term. Section 411 deals with the receiving of stolen property, stating that whoever knowingly receives or retains stolen property can be punished with up to three years of imprisonment, a fine, or both. Similarly, Tamil Nadu has a state amendment to this section as well, adding a sub-section (2) that specifies punishment for receiving stolen idols or icons from a place o

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-en-hi'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

text = "Hello, how are you?"
input_ids = tokenizer.encode(text, return_tensors="pt")
output = model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)

translated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(translated_text)

In [40]:
pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [41]:
import os
import json
from deep_translator import GoogleTranslator

In [44]:
translator = GoogleTranslator(source='auto', target='hi')

In [48]:
print(translator.translate('''According to the Indian Penal Code (IPC),
Section 441 states that whoever enters into or upon property in the possession of another with intent to commit an offence or to intimidate, 
insult or annoy any person in possession of such property, or having lawfully entered into or upon such property, unlawfully remains there 
with intent thereby to intimidate, insult or annoy any such person, or with intent to commit an offence, is said to commit "criminal trespass".
Therefore, building a house on land assigned for a shop in a colony without proper authorization would be considered as criminal trespass and
is punishable under the IPC.It is important to obtain proper permission and follow the designated land use regulations to avoid any legal consequences.'''))

भारतीय दंड संहिता (आईपीसी) के अनुसार,
धारा 441 में कहा गया है कि जो कोई अपराध करने या डराने-धमकाने के इरादे से किसी दूसरे की संपत्ति में प्रवेश करता है या उसके कब्जे में है।
ऐसी संपत्ति पर कब्ज़ा करने वाले, या ऐसी संपत्ति में कानूनी रूप से प्रवेश करने वाले या उस पर अवैध रूप से रहने वाले किसी भी व्यक्ति का अपमान या परेशान करना
ऐसे किसी भी व्यक्ति को डराने, अपमानित करने या परेशान करने के इरादे से, या अपराध करने के इरादे से, "आपराधिक अतिचार" कहा जाता है।
इसलिए, उचित प्राधिकरण के बिना किसी कॉलोनी में दुकान के लिए आवंटित भूमि पर घर का निर्माण आपराधिक अतिचार माना जाएगा और
आईपीसी के तहत दंडनीय है। किसी भी कानूनी परिणाम से बचने के लिए उचित अनुमति प्राप्त करना और निर्दिष्ट भूमि उपयोग नियमों का पालन करना महत्वपूर्ण है।


In [8]:
#!pip install -U sentence-transformers

from scipy.spatial import distance
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample sentence
sentences = ["When both of you and your sons are interested in staying with your errant husband despite his mental and physical tortures, you may consult with a family counselor or psychiatrist to resolve the issue in-house before opting for legal remedies that involve more complexities and lengthy processes. Sometimes, proper counseling or mediation resolves the issues faster and more smoothly than litigation."]

test="According to Section 498A of the Indian Penal Code, any husband or relative of the husband who subjects a woman to cruelty shall be punished with imprisonment for a term which may extend to three years and shall also be liable to fine. This includes mental as well as physical torture. You can file a complaint with the police and they will take necessary action. You can also seek help from a lawyer to file for divorce or seek protection under the Protection of Women from Domestic Violence Act, 2005. It is important to document any evidence of the torture and seek help as soon as possible."

print('Test sentence:',test)
test_vec = model.encode([test])[0]


for sent in sentences:
	similarity_score = 1-distance.cosine(test_vec, model.encode([sent])[0])
	print(f'\nFor {sent}\nSimilarity Score = {similarity_score} ')


Test sentence: According to Section 498A of the Indian Penal Code, any husband or relative of the husband who subjects a woman to cruelty shall be punished with imprisonment for a term which may extend to three years and shall also be liable to fine. This includes mental as well as physical torture. You can file a complaint with the police and they will take necessary action. You can also seek help from a lawyer to file for divorce or seek protection under the Protection of Women from Domestic Violence Act, 2005. It is important to document any evidence of the torture and seek help as soon as possible.

For When both of you and your sons are interested in staying with your errant husband despite his mental and physical tortures, you may consult with a family counselor or psychiatrist to resolve the issue in-house before opting for legal remedies that involve more complexities and lengthy processes. Sometimes, proper counseling or mediation resolves the issues faster and more smoothly t

In [115]:
import os
os.environ['OPENAI_API_KEY']='key'
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
import streamlit as st
from deep_translator import GoogleTranslator
translator = GoogleTranslator(source='auto', target='ur')

persist_directory='ip+pdf'
# from langchain_community.embeddings import OllamaEmbeddings
# embedding = OllamaEmbeddings()
embedding = OpenAIEmbeddings()

vectordb=Chroma(persist_directory=persist_directory,
                embedding_function=embedding)
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
# llm = OpenAI(temperature=0.3)
llm =Ollama(model="llama3.1")

## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Given the following context and a question, generate an answer based on this context only.
You're a chatbot specializing in Indian law, particularly the Indian Penal Code (IPC). Your user seeks guidance on legal matters within the scope of the IPC.
Engage with them in a helpful and informative manner, providing accurate answers to their questions and clarifications regarding relevant legal concepts ,statutes.
Ensure your responses are clear, concise, and tailored to the specific queries posed.


In the answer try to provide as much text as possible from the source document context .
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.
<context>
{context}
</context>
Question: {input}
"""
)
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

retriever=vectordb.as_retriever()

from langchain.chains import create_retrieval_chain


In [116]:
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [117]:
response=retrieval_chain.invoke({"input":"What is a bail? What is anticipatory bail? How do I get a bail? How much I have to pay for a bail? Is that amount refundable? Can I take bail in advance if I know somebody is going to take a legal action against me (I did not do anything wrong)? How long a bail is valid? Does it have some expiry?"})

In [118]:
response

{'input': 'What is a bail? What is anticipatory bail? How do I get a bail? How much I have to pay for a bail? Is that amount refundable? Can I take bail in advance if I know somebody is going to take a legal action against me (I did not do anything wrong)? How long a bail is valid? Does it have some expiry?',
 'context': [Document(metadata={'page': 839, 'source': 'Indian Penal Code Book.pdf'}, page_content='without sureties, fails without suﬃcient cause (the burden of proving which shall lie\nupon him), to appear in Court in accordance with the terms of the bail or bond, shall\nbe punished with imprisonment of either description for a term which may extend to\none year, or with ﬁne, or with both.\nExplanation.—The punishment under this section is—\n(a)in addition to the punishment to which the offender would be liable on a\nconviction for the offence with which he has been charged; and\n(b)without prejudice to the power of the Court to order forfeiture of the bond].\nCOMMENT.—\nCr PC, 

In [119]:
import pandas as pd
df=pd.read_csv("/Users/Downloads/qt1.csv")

In [120]:
df

,question,ground_truth,answer,contexts
0,Can someone build a house on land assigned for...,"In a residential colony, if a place is assigne...",NaN,NaN
1,Hi sir/Madam My husband always give me mentall...,When both of you and your sons are interested ...,NaN,NaN
2,"Sir, I worked in a RMC company as a third part...","Dear client, first contact the employer and as...",NaN,NaN
3,Someone hit my car from behind but I moved awa...,"It may be noted that currently, hit-and-run ca...",NaN,NaN
4,What is a bail? What is anticipatory bail? How...,Bail is a release of person who is accused of ...,NaN,NaN
5,My wife has filed a false divorce case against...,As far as your father and sisters are concerne...,NaN,NaN
6,How can file a Complaint in Consumer Court on ...,"This case do not attract Consumer Forum, & it'...",NaN,NaN
7,what can be done legally if someone is not ret...,suit for recovery can be filed.,NaN,NaN
8,can a sister have any claim on brothers salary...,In case the assets are joint and not divided s...,NaN,NaN
9,if Father is living and can a daughter demand ...,if it is an ancestral property then yes and if...,NaN,NaN


In [78]:
df["question"][5:]

5     We both are having difference. Can I move abro...
6     In India my mother want to gift me property bu...
7     Something has happened with me today, i was dr...
8     My wife has filed a false divorce case against...
9     How can file a Complaint in Consumer Court on ...
10    what can be done legally if someone is not ret...
11    can a sister have any claim on brothers salary...
12    if Father is living and can a daughter demand ...
13    Can a girlfriend of a man get custody rights t...
14            How we can file a suit for child custody?
15       How to evict a tenant who did not gave my rent
16    Can I cancel settlement deed for my minor chil...
17    My friend insulted his lady relative by callin...
18    Is it necessary that a private limited company...
19     time limitation to file case for probate of will
20    Is Identity Card Number of Next of Kin is mand...
21    Salam sir due to my mistake I had paid late in...
22    Sir I have purchased house from builders. 

In [121]:
a=df["question"]

In [122]:
df["question"]

0     Can someone build a house on land assigned for...
1     Hi sir/Madam My husband always give me mentall...
2     Sir, I worked in a RMC company as a third part...
3     Someone hit my car from behind but I moved awa...
4     What is a bail? What is anticipatory bail? How...
5     My wife has filed a false divorce case against...
6     How can file a Complaint in Consumer Court on ...
7     what can be done legally if someone is not ret...
8     can a sister have any claim on brothers salary...
9     if Father is living and can a daughter demand ...
10            How we can file a suit for child custody?
11       How to evict a tenant who did not gave my rent
12    Can I cancel settlement deed for my minor chil...
13    My friend insulted his lady relative by callin...
14    Is it necessary that a private limited company...
15    Is Identity Card Number of Next of Kin is mand...
16    Sir I have purchased house from builders. Some...
17    Relative found a 2 floored rented shop thr

In [123]:
j=0
for i in a:
    response=retrieval_chain.invoke({"input":i})
    df['answer'][j]=response['answer']
    df['contexts'][j]=response['context']
    j+=1

/var/folders/__/flq7sb_j7l93199xmqjbt3fh0000gn/T/ipykernel_11213/4284119625.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['answer'][j]=response['answer']
/var/folders/__/flq7sb_j7l93199xmqjbt3fh0000gn/T/ipykernel_11213/4284119625.py

In [124]:
df

,question,ground_truth,answer,contexts
0,Can someone build a house on land assigned for...,"In a residential colony, if a place is assigne...","Based on the provided context, I must state th...",[page_content='the Legislature in framing sect...
1,Hi sir/Madam My husband always give me mentall...,When both of you and your sons are interested ...,"I can't provide legal advice, but I can offer ...",[page_content='were persistently torturing her...
2,"Sir, I worked in a RMC company as a third part...","Dear client, first contact the employer and as...",I don't see any relevant information regarding...,[page_content='Where a conviction under sectio...
3,Someone hit my car from behind but I moved awa...,"It may be noted that currently, hit-and-run ca...","Based on the context provided, it seems that y...",[page_content='truck against the cyclist and t...
4,What is a bail? What is anticipatory bail? How...,Bail is a release of person who is accused of ...,Given the context of the Indian Penal Code (IP...,"[page_content='without sureties, fails without..."
5,My wife has filed a false divorce case against...,As far as your father and sisters are concerne...,I cannot provide legal advice. If you are expe...,[page_content='an offence under this section.3...
6,How can file a Complaint in Consumer Court on ...,"This case do not attract Consumer Forum, & it'...",I don't know.,[page_content='said ﬁxed deposit adjusted a pa...
7,what can be done legally if someone is not ret...,suit for recovery can be filed.,"Based on the provided context, I'd like to add...","[page_content='take place. The Court has, howe..."
8,can a sister have any claim on brothers salary...,In case the assets are joint and not divided s...,I don't know.\n\nThe provided context does not...,[page_content='wherefrom the ingredients of an...
9,if Father is living and can a daughter demand ...,if it is an ancestral property then yes and if...,I don't know.\n\nThe provided context discusse...,[page_content='list of ornaments and other hou...


In [125]:
df.to_csv("final_ollama3.1_only_llm.csv")